In [1]:
import json

import geopandas as gpd
import pandas as pd

from hidromet.ANA import ANA
from hidromet import config
from hidromet import limpeza
from hidromet import modelos
from pathlib import Path
from hidromet import contornos

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Obter inventário de postos da ANA

In [2]:
ana = ANA()
inventario_plu = ana.inventario(tipoest=2)
inventario_plu.loc[:, ["latitude", "longitude"]] = inventario_plu.loc[:, ["latitude", "longitude"]].apply(pd.to_numeric)

# Recortar postos dentro do contorno das bacias

## Postos da ANA dentro dos contornos

In [3]:
gdf = gpd.read_file(config.dir_contorno)

coordenadas = contornos.zipar_coordenadas(latitude=inventario_plu.latitude, longitude=inventario_plu.longitude)
postos_plu = contornos.criar_geodataframe(coordenadas=coordenadas, df=inventario_plu)

geometria = gdf.iloc[0].geometry
bacia = gdf.iloc[0].bacia
postos = postos_plu.geometry

postos_ana_dentro = contornos.recortar_postos(postos=postos, contorno=geometria)

## Obter séries de chuva por posto

In [4]:
arquivo_bacia = config.dir_prec_ana.joinpath(f"{bacia}.csv")
data_inicial = '01/01/2000'

bacia_df = limpeza.remover_codigos_duplicados(postos_ana_dentro)
n_postos = len(bacia_df.index)

chuvas_bacia = list()
postos_sem_dado = list()
postos_com_falha = list()
postos_nao_representativos = list()
postos_ok = list()
for posto in bacia_df.index:
    coordenadas = bacia_df.loc[posto, 'geometry']
    df = ana.obter_chuva(cod_estacao=posto, data_inicial=data_inicial)
    df.index = pd.to_datetime(df.index)

    df = limpeza.remover_duplicados(df)
    df[posto] = limpeza.remover_outliers(df[posto])
    df[posto] = limpeza.substituir_negativos(df[posto])
    falhas = limpeza.contabilizar_falhas(df[posto])
    df = limpeza.remover_meses_nao_representativos(df)
    anos_disponiveis = limpeza.anos_disponiveis(df)
    n_dados = len(df)


    info_posto = modelos.Posto(
        latitude=coordenadas.y,
        longitude=coordenadas.x,
        codigo=posto,
        n_dados=n_dados
    )

    if not df.empty and falhas < 0.15 and anos_disponiveis > 8:
        postos_ok.append(info_posto)
        chuvas_bacia.append(df)
    elif df.empty:
        postos_sem_dado.append(info_posto)
    elif anos_disponiveis < 8: 
        postos_nao_representativos.append(info_posto)
    elif falhas > 0.15:
        postos_com_falha.append(info_posto)


info_bacia = modelos.Bacia(
    serie_vazia=postos_sem_dado,
    nao_representativos=postos_nao_representativos,
    com_falhas=postos_com_falha,
    postos_ok=postos_ok,
    n_postos=n_postos,
    bacia=bacia
)


df_chuva = pd.concat(chuvas_bacia, axis=1)
df_chuva.to_csv(arquivo_bacia)

arquivo_json = Path(config.dir_prec_ana, f'coords_{bacia}.json')
with open(arquivo_json, 'w') as f:
    json.dump(info_bacia.dict(), f)